# Rounding In Python — When Arithmetic Isn’t Quite Right
A Python Notebook to accompany the article [Rounding In Python — When Arithmetic Isn’t Quite Right]().

Refer to [this guide](https://medium.com/k-folds/saving-without-gitting-downloading-from-github-without-using-git-72b90c694af4) for help downloading the notebook!


Rounding in Python is easy, just call `round(number_to_round, significant_digits)`. If you don't specify significant digits, it rounds to an integer.

In [1]:
print(round(1.5))
print(round(2.1, 0))

2
2.0


But, not all rounding acts like you'd expect!


In [2]:
print(round(2.5))

2


Why???

Rounding gets weird because rounding in Python, and any programming language, happens in binary. The number gets converted to binary, rounded, then converted back into a base ten number for display.

### A Binary Refresher
Instead of grouping by units of ten, binary groups in units of 2. Every significant digit represents 2 raised to that power.

To see what a number is in binary, call bin(number).

In [3]:
print("0: " + str(bin(0)))
print("1: " + str(bin(1)))
print("2: " + str(bin(2)))
print("3: " + str(bin(3)))
print("4: " + str(bin(4)))
print("5: " + str(bin(5)))
print("6: " + str(bin(6)))

0: 0b0
1: 0b1
2: 0b10
3: 0b11
4: 0b100
5: 0b101
6: 0b110


The results are printed out with a '0b' in front to indicate the string is a binary string.

We see 0 is still 0, but now we add a significant digit when we reach another power of 2.

| Base Ten Number  | Powers of 2  |  Binary Number |
|---|---|---|
| 1  |  2^0 |  1 |
| 2  |  2^1 |  10 |
| 3 |  2 + 1 = 2^1 + 1|  11 |
| 4 |  2^2|  100 |
| 5 |  4 + 1 = 2^2 + 1|  101 |
| 6 |  4 + 2 = 2^2 + 2^1 |  110 |
| 7 |  4 + 2 + 1= 2^2 + 2^1 + 2^0 |  111 |


And so on...

## Using Decimal
To have more control over rounding, use the Decimal package. 

To create a Decimal number, call decimal.Decimal(number)

In [4]:
import decimal

number = decimal.Decimal(1.1)
print(number)

1.100000000000000088817841970012523233890533447265625


It takes the current number stored in Python and converts that to a decimal object. Since 1.1 cannot be stored perfectly in binary, the number is already corrupted before it's even passed to Decimal. To fix this, always send the number as a string!

In [5]:
number = decimal.Decimal('1.1')
print(number)

1.1


Decimal numbers interact perfectly with regular Python numbers, so you can still add/subtract/multiply to your heart's content. We can even use Python's round function with it to round numbers.

In [6]:
print(number + 1)
print(number * 5)
print(round(number))

2.1
5.5
1


But the round function still operates under Python's default rounding rules, so there will still be problems.

In [7]:
number = decimal.Decimal('2.5')
print(round(number))

2


## Implementing Custom Rounding
To see Decimal's rounding settings call the getcontext() method.

In [8]:
print(decimal.getcontext())

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[FloatOperation], traps=[InvalidOperation, DivisionByZero, Overflow])


Even decimal defaults to ROUND_HALF_EVEN like Python's default... But we can change that. To modify the way calculations are rounded, change the setting like so:

In [9]:
decimal.getcontext().rounding = 'ROUND_HALF_UP'
print(decimal.getcontext())

Context(prec=28, rounding=ROUND_HALF_UP, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[FloatOperation], traps=[InvalidOperation, DivisionByZero, Overflow])


This makes sure that if you have truncation in the lower decimal places, the result is rounded up. Which helps with really small numbers, but doesn’t help when rounding a single number.

The rounding functionality is a little different, you have to use the quantize function. The function rounds a number to the specified significant digit, and applies the rounding rule you specify if you don’t want to use the current one specified getcontext().

In [10]:
number = decimal.Decimal('1.1')
rounded = number.quantize(decimal.Decimal('0.01'))
print(rounded)

1.10


In [11]:
number = decimal.Decimal('1.1')
rounded = number.quantize(decimal.Decimal('1'), rounding='ROUND_HALF_UP')
print(rounded)

1


In [12]:
number = decimal.Decimal('2.5')
rounded = number.quantize(decimal.Decimal('1'), rounding='ROUND_HALF_UP')
print(rounded)

3


The quantize function is handy, but not as convenient to use. The function round_decimal() below mimics Python's built in rounding function, making the package easier to use!

It takes any number, converts it to a string, then converts it to a decimal object to make sure the number get’s represented exactly. It then calls quantize() depending on the rounding specified, and returns a floating point or integer as the result:

In [13]:
def round_decimal(x, digits = 0):
    #casting to string then converting to decimal
    x = decimal.Decimal(str(x))
    
    #rounding for integers
    if digits == 0:
        return int(x.quantize(decimal.Decimal("1"), rounding='ROUND_HALF_UP'))

    #string in scientific notation for significant digits: 1e^x 
    if digits > 1:
        string =  '1e' + str(-1*digits)
    else:
        string =  '1e' + str(-1*digits)

    #rounding for floating points
    return float(x.quantize(decimal.Decimal(string), rounding='ROUND_HALF_UP'))

In [14]:
print("Built-in Rounding:")
print(round(555.555))
print(round(555.555, 1))
print(round(555.555, -2))

print("Custom Rounding:")
print(round_decimal(555.555))
print(round_decimal(555.555, 1))
print(round_decimal(555.555, -2))


Built-in Rounding:
556
555.6
600.0
Custom Rounding:
556
555.6
600.0


In [15]:
print("Built-in Rounding:")
print(round(2.5))

print("Custom Rounding:")
print(round_decimal(2.5))

Built-in Rounding:
2
Custom Rounding:
3
